# Text Wrangling and Regex

Working with text: applying string methods and regular expressions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import pandas as pd

## Demo 1: Canonicalizing County Names

Load both **county_and_state.csv** and **county_and_population.csv**

In [ ]:
# display both frames
#code Here



,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


Both of these DataFrames share a "County" column. Unfortunately, formatting differences mean that we can't directly merge the two DataFrames using the "County"s.

In [1]:
#merge both frames usding pd.merge
#The output would only show the header. We will resolve it later on.

To address this, we can **canonicalize** the "County" string data to apply a common formatting.

In [ ]:
def canonicalize_county(county_series):   #take help from your slides to cover this part
                          # lowercase
                           # remove space
                           # replace &
                           # remove dot
                           # remove "county"
                           # remove "parish"


Apply canonicalize_county on 'County' columns in both frames.

In [ ]:
# Code Here
df1['County'] = canonicalize_county(df1['County'])
df2['County'] = canonicalize_county(df2['County'])

Now, the merge works as expected!

In [ ]:
# code Here
#now merge df1 and df2

,County,Population,State
0,dewitt,16798,IL
1,lacquiparle,8067,MN
2,lewisandclark,55716,MT
3,stjohnthebaptist,43044,LS


## Demo 2: Extracting Log Data

Load log.txt

In [ ]:
#code Here
        # Process each line as needed
#Hint: the file will be read almost the same way as we read the json file.
#Use the file.readlines function to read the lines in the txt file

data

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"']

Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. That is, slicing by some fixed offset isn't going to work.

Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [ ]:
# code here
data[0]

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"'

Apply string functions of python to extract date from first entry in log file.

In [ ]:
# you might take help from the slides
                                        # find the text enclosed in square brackets
                                        # split up the date/month/year
                                        # split up the hour:minute:second
                                        # split the timezone after the blank space


('26', 'Jan', '2014', '10', '47', '58 ')

This worked, but felt fairly "hacky" – the code above isn't particularly elegant. A much more sophisticated but common approach is to extract the information we need using a *regular expression*.


# Regular Expressions


## String Extraction with Regex

Python `re.findall` returns a list of all extracted matches: extract numbers from given string.

In [ ]:
import re

text = "My social security number is 123-45-6789 bro, or actually maybe it’s 321-45-6789.";
# code Here

#Expected output:  ['123-45-6789', '321-45-6789']

<br/>

Now, let's see vectorized extraction in `pandas`:

 `.str.findall` returns a `Series` of lists of all matches in each record.

In [ ]:
data = ['987-65-4321','forty','123-45-6789 bro or 321-45-6789','999-99-9999']
# code here to convert the above data into a dataframe

,SSN
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


Find all entries containing numbers in dataframe.

In [2]:
# -> Series of lists
# code Here



#Expected Output:
#  0                   [987, 65, 4321]
#  1                                []
#  2    [123, 45, 6789, 321, 45, 6789]
#  3                   [999, 99, 9999]
#  Name: SSN, dtype: object

## Extraction Using Regex Capture Groups

The Python function `re.findall`, in combination with parentheses returns specific substrings (i.e., **capture groups**) within each matched string, or **match**.

In [ ]:
text = """I will meet you at 08:30:00 pm tomorrow"""
# code here

[('08', '30', '00')]

In [ ]:
# the three capture groups in the first matched string
hour, minute, second = matches[0]

<br/>

In `pandas`, we can use `.str.extract` to extract each capture group of **only the first match** of each record into separate columns.

In [ ]:
# back to SSNs
df_ssn

,SSN
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


In [ ]:
# Will extract the first match of all groups
# code here

,0,1,2
0,987,65,4321
1,NaN,NaN,NaN
2,123,45,6789
3,999,99,9999


Alternatively, `.str.extractall` extracts **all matches** of each record into separate columns. Rows are then MultiIndexed by original record index and match index.

In [ ]:
# -> DataFrame, one row per match
# code Here

0   1     2
  match               
0 0      987  65  4321
2 0      123  45  6789
  1      321  45  6789
3 0      999  99  9999

## Canonicalization with Regex

In regular Python, canonicalize with `re.sub` (standing for "substitute"):

In [ ]:
text = '<div><td valign="top">Moo</td></div>'

#code here to extract the word Moo

<br/>

In `pandas`, canonicalize with `Series.str.replace`.

In [ ]:
# example dataframe of strings, convert in dataframe
df_html = ['<div><td valign="top">Moo</td></div>',
                   '<a href="http://ds100.org">Link</a>',
                   '<b>Bold text</b>']
df_html

,Html
0,"<div><td valign=""top"">Moo</td></div>"
1,"<a href=""http://ds100.org"">Link</a>"
2,<b>Bold text</b>


In [ ]:
# Series -> Series
#Extract only words from the above given df_html like Moo, Link and Bold text



# Revisiting Text Log Processing using Regex

### Python `re` version

In [ ]:
line = log_lines[0]
display(line)
# code here

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

('26', 'Jan', '2014', '10', '47', '58', '-0800')

### `pandas` version

In [ ]:
# code here

,Log
0,169.237.46.168 - - [26/Jan/2014:10:47:58 -0800...
1,"193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] ""..."
2,"169.237.46.240 - """" [3/Feb/2006:10:18:37 -0800..."


Option 1: `Series.str.findall`

In [ ]:
# code here

0    [(26, Jan, 2014, 10, 47, 58, -0800)]
1      [(2, Feb, 2005, 17, 23, 6, -0800)]
2     [(3, Feb, 2006, 10, 18, 37, -0800)]
Name: Log, dtype: object

<br/>

Option 2: `Series.str.extractall`

In [ ]:
# code here

,,0,1,2,3,4,5,6
,match,,,,,,,
0,0,26,Jan,2014,10,47,58,-0800
1,0,2,Feb,2005,17,23,6,-0800
2,0,3,Feb,2006,10,18,37,-0800


Wrangling either of these two DataFrames into a nice format (like below) is left as an exercise for you!


||Day|Month|Year|Hour|Minute|Second|Time Zone|
|---|---|---|---|---|---|---|---|
|0|26|Jan|2014|10|47|58|-0800|
|1|2|Feb|2005|17|23|6|-0800|
|2|3|Feb|2006|10|18|37|-0800|


In [ ]:
# your code here
...